# Описание задачи

В этом задании необходимо достичь максимального качества предсказания, используя навыки полученные за 4 недели обучения

Кто достигнет максимального значения на тестовой выборке, получит *15 баллов*

Пожалуйста, оформляйте ноутбук аккуратно. Все выводы подписывайте, оформляйте заголовки.

Для справедливой оценки все должны использовать одну и ту же часть исходного датасета в качестве тестового набора данных. Используйте разбиение приведенное ниже


```
train_test_split(X, y, test_size = 0.3, random_state = 69)
```
И модель, с которой вы работаете - это линейная регрессия. Другие алгоритмы не используйте.

Метрика, которую вы должны максимизировать

```
r2_score(y_test, y_pred) и root_mean_squared_error(y_test, y_pred)
```

Целевая переменная - *count*

Обратите внимание на столбцы - 'casual', 'registered'. Эти столбцы в сумме дают целевую переменную, по этой причине их надо удалить. Они линейно зависимы.




Используйтие средства библиотеки sklearn, внимательно изучите ее. Существует огромное количество методов для выполнения каждого из этапов, не бойтесь использовать методы неприведенные в лекции

Этапы, которые необходимо проделать в работе для успешного достижения результата:
* EDA (Исследовать данные, понять с чем имеете дело, наработать идеи для генерации фичей и их обработки, обязательно корреляционные матрицы и графики с hue=классы объектов)
* Анализ выбросов и их обработка (в задачах классификации выброс - это объект с таким признаковым описанием, которое отличается очень сильно от типичного и больше соответствует другому классу)
* Генерация новых фичей
* Обработка пропусков (вы можете сгенерировать новые фичи с пропусками, тогда надо подумать об их обработке)
* Обработка категориальных признаков
* Масштабирование вещественных признаков
* Трансформирование таргета
* Нелинейные автоматические трансформации признаков (Kernel Trick, Transfomers)
* Подбор параметров модели (можете менять не только константы, но и оптимизационные алгоритмы и методы регуляризации)
* Отбор признаков
* Обязательно: Составление пайплайна обучения
* Обязательно: Постройте график зависимости y_pred от y_true в разбиении на test и train. Предсказание идеального алгоритма даст прямую y=x, посмотрите какой график получается у вас.

# Bike Sharing Demand
По историческим данным о прокате велосипедов и погодным условиям необходимо оценить спрос на прокат велосипедов.

В наборе признаков присутсвуют вещественные, категориальные, и бинарные данные.

### Библиотеки

In [132]:
%pylab inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import TransformedTargetRegressor

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\Leoni\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\pylab.py:166: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


### Загрузка данных

In [133]:
df = pd.read_csv('bike_sharing_demand.csv')
df["datetime"] = pd.to_datetime(df["datetime"])
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


***datetime*** - hourly date + timestamp  

***season*** -  1 = spring, 2 = summer, 3 = fall, 4 = winter

***holiday*** - whether the day is considered a holiday

***workingday*** - whether the day is neither a weekend nor holiday

***weather*** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
    
***temp*** - temperature in Celsius

***atemp*** - "feels like" temperature in Celsius

***humidity*** - relative humidity

***windspeed*** - wind speed

***casual*** - number of non-registered user rentals initiated

***registered*** - number of registered user rentals initiated

***count*** - number of total rentals

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


Рассмотрим корреляцию признаков

In [135]:
df[['temp','atemp','humidity','windspeed','count', 'casual', 'registered']].corr()

,temp,atemp,humidity,windspeed,count,casual,registered
temp,1.000000,0.984948,-0.064949,-0.017852,0.394454,0.467097,0.318571
atemp,0.984948,1.000000,-0.043536,-0.057473,0.389784,0.462067,0.314635
humidity,-0.064949,-0.043536,1.000000,-0.318607,-0.317371,-0.348187,-0.265458
windspeed,-0.017852,-0.057473,-0.318607,1.000000,0.101369,0.092276,0.091052
count,0.394454,0.389784,-0.317371,0.101369,1.000000,0.690414,0.970948
casual,0.467097,0.462067,-0.348187,0.092276,0.690414,1.000000,0.497250
registered,0.318571,0.314635,-0.265458,0.091052,0.970948,0.497250,1.000000


Удалим casual и registered т.к они в сумме дают count, а также atemp т.к он зависит от temp и имеет с count меньшую корреляцию.

In [136]:
df = df.drop(['casual', 'registered', 'atemp'], axis=1)
df

,datetime,season,holiday,workingday,weather,temp,humidity,windspeed,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,81,0.0000,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,80,0.0000,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,80,0.0000,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,75,0.0000,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,75,0.0000,1
...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,50,26.0027,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,57,15.0013,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,61,15.0013,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,61,6.0032,129


Разобьём датасет

In [137]:
X = df.drop('count', axis=1)
y = df['count']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 69)
train = pd.concat([X_train, y_train], axis=1)
train

,datetime,season,holiday,workingday,weather,temp,humidity,windspeed,count
6867,2012-04-04 11:00:00,2,0,1,2,24.60,43,23.9994,213
5296,2011-12-14 18:00:00,4,0,1,2,14.76,62,11.0014,390
8897,2012-08-13 02:00:00,3,0,1,1,26.24,69,7.0015,9
7773,2012-06-04 06:00:00,2,0,1,1,21.32,68,0.0000,139
355,2011-01-16 07:00:00,1,0,0,2,9.84,56,19.9995,3
...,...,...,...,...,...,...,...,...,...
8631,2012-08-02 00:00:00,3,0,1,1,28.70,74,0.0000,63
9818,2012-10-13 11:00:00,4,0,0,1,17.22,35,8.9981,527
10859,2012-12-18 21:00:00,4,0,1,1,14.76,50,15.0013,168
4041,2011-09-19 10:00:00,3,0,1,2,22.96,64,8.9981,105


Удалим выбросы по квантилям

In [138]:
for feature in ['temp', 'humidity', 'windspeed']:
  Q1 = X_train[feature].quantile(0.05)
  Q3 = X_train[feature].quantile(0.95)
  train = train[(train[feature] >= Q1) & (train[feature] <= Q3)]

X_train = train[X_train.columns]
y_train = train[y_train.name]
train

,datetime,season,holiday,workingday,weather,temp,humidity,windspeed,count
6867,2012-04-04 11:00:00,2,0,1,2,24.60,43,23.9994,213
5296,2011-12-14 18:00:00,4,0,1,2,14.76,62,11.0014,390
8897,2012-08-13 02:00:00,3,0,1,1,26.24,69,7.0015,9
7773,2012-06-04 06:00:00,2,0,1,1,21.32,68,0.0000,139
355,2011-01-16 07:00:00,1,0,0,2,9.84,56,19.9995,3
...,...,...,...,...,...,...,...,...,...
8631,2012-08-02 00:00:00,3,0,1,1,28.70,74,0.0000,63
9818,2012-10-13 11:00:00,4,0,0,1,17.22,35,8.9981,527
10859,2012-12-18 21:00:00,4,0,1,1,14.76,50,15.0013,168
4041,2011-09-19 10:00:00,3,0,1,2,22.96,64,8.9981,105


Выделим части datetime в отдельные поля, добавим признаки peak_hour, peak_season
Класс пайплайна для добавления фич:

In [139]:
class FeatureTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        df = X.copy()

        df["hour"] = df["datetime"].dt.hour
        df["day"] = df["datetime"].dt.day
        df['month'] = df["datetime"].dt.month
        df["peak_season"] = df["season"].apply(lambda x: 1 if x in [1, 2] else 0)
        df['peak_hour'] = df['hour'].apply(lambda x: 1 if x in [7, 8, 9, 17, 18, 19] else 0)
        df["bad_weather"] = df["weather"].apply(lambda x : 1 if x in [3,4] else 0)
        df = df.drop(columns=["datetime", "weather", "season"])

        return df

Нормализация + one-hot encoding:

In [140]:
preprocessor = ColumnTransformer(
    [
      ('numeric', MinMaxScaler(), ["temp", "humidity", "windspeed"]),
      ('categorical', OneHotEncoder(handle_unknown='ignore', sparse_output=False), 
       ['holiday', 'workingday', 'hour', 'day', 'month', 'bad_weather', "peak_hour", "peak_season"])
    ]
)

Пайплайн

In [141]:
pipeline = Pipeline(
    [
        ('transformer', FeatureTransformer()),
        ('preprocessor', preprocessor),
        ('poly', PolynomialFeatures(2)),
        ('regressor', Ridge())
    ]
)

Применим пайплайн для логарифмированного таргета:

In [142]:
target_transformer = TransformedTargetRegressor(regressor=pipeline, func=np.log1p, inverse_func=np.expm1)

param_grid = {'regressor__regressor__alpha': [0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.08, 1, 2, 3, 5, 8, 10, 20, 50, 100]}

clf = GridSearchCV(target_transformer, param_grid, cv = 5)

clf.fit(X_train, y_train)

test_pred = clf.predict(X_test)
r2_test = r2_score(y_test, test_pred)
root_test = root_mean_squared_error(y_test, test_pred)
print(f"Test r2: {r2_test}, test rmse: {root_test}")

train_pred = clf.predict(X_train)
r2_train = r2_score(y_train, train_pred)
root_train = root_mean_squared_error(y_train, train_pred)
print(f"Train r2: {r2_train}, train rmse: {root_train}")

Test r2: 0.8337195781524305, test rmse: 74.00974656590199
Train r2: 0.8816127199998578, train rmse: 61.859446521183244
